In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import toolsdiadem.tools as dia

In [95]:
pi = np.pi
E0 = 1
f0 = 10e9
omega0 = 2 * np.pi * f0
lambda0 = c / f0
k0 = 2 * np.pi / lambda0

$ E_{a_y} = E_0 cos \left( \frac{\pi}{A}x \right) e^{ -j \frac{\beta}{2} \left( \frac{x^2}{R_0^{E^2}}  + \frac{y^2}{R_0^{H^2}} \right) } $

In [304]:
c = 3e8
A = 0.109
B = 0.079
R = 0.245
a = 0.0229 # WR90 WG16
b = 0.0102 # WR90 WG16
dx = lambda0 / 2
dy = lambda0 / 2
theta_E = np.arctan( (A-a)/2 / R )
theta_H = np.arctan( (B-b)/2 / R )
print( f"theta_E = {theta_E*180/3.14:.2f}, theta_H = {theta_H*180/3.14:.2f}" )
R0_E = A / 2 / np.sin(theta_E)
R0_H = B / 2 / np.sin(theta_H)
beta0 = 2 * np.pi / lambda0
beta =  beta0 * ( 1 - (lambda0 / (2*a))**2 )**0.5 # propagation constant of the TE10 mode
print( f"R0_E = {R0_E:.2f}, R0_H = {R0_H:.2f}" )
x = np.concatenate( 
    (np.flip(np.arange(0, -A/2, -dx)[1:],0), 
              np.arange(0, A/2, dx)) ).reshape(1,-1)
y = np.concatenate( 
    (np.flip(np.arange(0, -B/2, -dy)[1:],0), 
              np.arange(0, B/2, dy)) ).reshape(-1,1)
Nx = x.shape[1]
Ny = y.shape[0]
print(f"Nx = {Nx}, Ny = {Ny}")

theta_E = 9.97, theta_H = 8.00
R0_E = 0.31, R0_H = 0.28
Nx = 7, Ny = 5


The phase error occurs due to the difference between the length from the center of the feeder to the center of the horn aperture and the length from the center of the feeder to the horn edge.  This complicates the analysis, and makes the results for the waveguide apertures invalid.

The tangential fields arriving at the input of the horn are the transverse field components of the waveguide dominant mode TE10.

Huygens source (Balanis p.807).

In [305]:
# beta is the waveguide transverse impedance
E_a_y = E0 * np.cos(np.pi/A * x) * np.exp( -1j * beta / 2 * (x**2/R0_E**2 + y**2/R0_H**2) )

In [306]:
hires = 1

In [307]:
pointsToAdd = 100
E_a_y_extended = np.zeros((E_a_y.shape[0]+2*pointsToAdd, E_a_y.shape[1]+2*pointsToAdd), dtype=complex)
E_a_y_extended[pointsToAdd:pointsToAdd+Ny, pointsToAdd:pointsToAdd+Nx] = E_a_y[:,:]
hiresNx = Nx + 2 * pointsToAdd
hiresNy = Ny + 2 * pointsToAdd

In [308]:
if hires:
    kx = np.fft.fftshift(np.fft.fftfreq(hiresNx,dx)) * 2 * np.pi
    ky = np.fft.fftshift(np.fft.fftfreq(hiresNy,dy)) * 2 * np.pi
else:
    kx = np.fft.fftshift(np.fft.fftfreq(Nx,dx)) * 2 * np.pi
    ky = np.fft.fftshift(np.fft.fftfreq(Ny,dy)) * 2 * np.pi

mesh_kx, mesh_ky = np.meshgrid(kx, ky)

validIdx = np.where( k0**2 >= (mesh_kx**2 + mesh_ky**2) )
nonValidIdx = np.where( k0**2 < (mesh_kx**2 + mesh_ky**2) )
kz = np.zeros(mesh_kx.shape)
kz[validIdx] = np.power( k0**2 - mesh_kx[validIdx]**2 - mesh_ky[validIdx]**2, 0.5 )

In [309]:
theta, phi = dia.getThetaPhi(mesh_kx, mesh_ky, kz, k0)
u = np.sin(theta) * np.cos(phi)
v = np.sin(theta) * np.sin(phi)
w = np.cos(theta)

In [310]:
fig, (ax0, ax1) = plt.subplots(2,1)
idx_x = int(Nx/2)
idx_y = int(Ny/2)

ax0.plot(np.abs(E_a_y[:,idx_x]), label="H plane")
ax0.legend()
ax0.grid()

ax1.plot(np.abs(E_a_y[idx_y,:]), label="E plane")
ax1.legend()
ax1.grid()

# Orfanidis (18. Radiation from Apertures p.805)

In [311]:
if hires:
    fx = 0
    fy = np.fft.fftshift(np.fft.fft2(E_a_y_extended))
else:
    fx = 0
    fy = np.fft.fftshift(np.fft.fft2(E_a_y))

In [312]:
idx = np.where(theta==pi/2)
aux = np.zeros(theta.shape)
aux[idx] = 1
plt.figure()
plt.pcolor(aux)

In [313]:
r = 1
orfa_E_theta = 1j * k0 * np.exp(-1j*k0*r) / (2*pi*r)                 * (fx * np.cos(phi) + fy * np.sin(phi))
orfa_E_phi   = 1j * k0 * np.exp(-1j*k0*r) / (2*pi*r) * np.cos(theta) * (fy * np.cos(phi) - fx * np.sin(phi))
orfa_E_tot   = (orfa_E_theta**2 + orfa_E_phi**2)**0.5
orfa_E_tot_norm = orfa_E_tot / np.amax( np.abs(orfa_E_tot ) )
dBmin = -30
dBmax = 0

In [314]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolor(u, v, 20 * np.log10(np.abs(orfa_E_tot_norm)), vmin=dBmin, vmax=dBmax)
ax0.set_aspect("equal")
dia.addColorBar(im, ax0)

im = ax1.pcolor(kx, ky, 20 * np.log10(np.abs(orfa_E_tot_norm)), vmin=dBmin, vmax=dBmax)
ax1.set_aspect("equal")
dia.addColorBar(im, ax1)

In [315]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolor(u, v, theta, label="theta")
ax0.set_aspect("equal")
ax0.legend()
dia.addColorBar(im, ax0)

im = ax1.pcolor(u, v, phi, label="phi")
ax1.set_aspect("equal")
ax1.legend()
dia.addColorBar(im, ax1)

In [317]:
fig, ax = plt.subplots(1, 1)

idx_x = int(hiresNx/2)
idx_y = int(hiresNy/2)

ax.plot(v[:,idx_x]*180/pi, 20*np.log10(np.abs(orfa_E_tot_norm[:,idx_x])), '.-', label="H plane")
ax.plot(u[idx_y,:]*180/pi, 20*np.log10(np.abs(orfa_E_tot_norm[idx_y,:])), '.-', label="E plane")
ax.legend()
ax.grid()
ax.set_ylim(dBmin, dBmax)

(-30, 0)

In [286]:
hiresNx, hiresNy

(208, 206)

## Rectangular apertures

In [320]:
delta_theta_x = 0.886 * lambda0 / A *180 / pi
delta_theta_y = 0.886 * lambda0 / B *180 / pi
print(f"delta_theta_x = {delta_theta_x:.2f}, delta_theta_y = {delta_theta_y:.2f}")

delta_theta_x = 13.97, delta_theta_y = 19.28
